Creating fused raster of imagery and rasterized morphometrics, using mean, min, max zonal stats

In [37]:
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np

In [38]:
def stack_and_resample(
    satellite_tif,
    secondary_tif,
    output_tif,
    target_resolution=100
):
    with rasterio.open(satellite_tif) as sat, rasterio.open(secondary_tif) as sec:

        out_nodata = -9999.0

        sat_data = sat.read(masked=True).astype("float32").filled(out_nodata)
        sec_data = sec.read(masked=True).astype("float32").filled(out_nodata)

        merged = np.concatenate([sat_data, sec_data], axis=0)
        band_count = merged.shape[0]

        scale_x = target_resolution / sat.res[0]
        scale_y = target_resolution / sat.res[1]

        out_transform = sat.transform * sat.transform.scale(scale_x, scale_y)
        out_width = int(np.ceil(sat.width / scale_x))
        out_height = int(np.ceil(sat.height / scale_y))

        mean_out = np.full((band_count, out_height, out_width), out_nodata, dtype="float32")
        min_out  = np.full_like(mean_out, out_nodata)
        max_out  = np.full_like(mean_out, out_nodata)

        for b in range(band_count):
            for dst, method in (
                (mean_out[b], Resampling.average),
                (min_out[b],  Resampling.min),
                (max_out[b],  Resampling.max),
            ):
                reproject(
                    merged[b],
                    dst,
                    src_transform=sat.transform,
                    src_crs=sat.crs,
                    src_nodata=out_nodata,
                    dst_transform=out_transform,
                    dst_crs=sat.crs,
                    dst_nodata=out_nodata,
                    resampling=method,
                )

        final = np.concatenate([mean_out, min_out, max_out], axis=0)
        final = np.ascontiguousarray(final)  # IMPORTANT

        profile = {
            "driver": "GTiff",
            "width": out_width,
            "height": out_height,
            "count": final.shape[0],
            "dtype": "float32",
            "crs": sat.crs,
            "transform": out_transform,
            "nodata": out_nodata,
            "BIGTIFF": "YES",
        }

        with rasterio.open(output_tif, "w", **profile) as dst:
            for i in range(final.shape[0]):
                dst.write(final[i], i + 1)

    print("saved successfully")

In [57]:
### Berlin
# image
satellite_tif = r'imagery\berlin_20170519.tif'
# rasterized morphometrics from non-weighted model
secondary_tif = r'rasterized_morphometrics\berlin_rasterized_morphometrics_fold3.tif'
# output for fused image with morphometrics
output_tif = r'imagery\berlin_20170519_morphometrics_meanminmax_grid.tif'

## image
# satellite_tif = r'imagery\berlin_20170519.tif'
## rasterized morphometrics from weighted model
# secondary_tif = r'rasterized_morphometrics\berlin_rasterized_morphometrics_fold3_weighted.tif'
## output for fused image with morphometrics
# output_tif = r'imagery\berlin_20170519_morphometrics_weighted_meanminmax_grid.tif'

### Hong Kong
## image
#satellite_tif = r'imagery\hongkong_20180321.tif'
## rasterized morphometrics from non-weighted model
#secondary_tif = r'rasterized_morphometrics\hongkong_rasterized_morphometrics_fold3.tif'
## output for fused image with morphometrics
#output_tif = r'imagery\hongkong_20180321_morphometrics_meanminmax_grid.tif'

## image
# satellite_tif = r'imagery\hongkong_20180321.tif'
## rasterized morphometrics from weighted model
# secondary_tif = r'rasterized_morphometrics\hongkong_rasterized_morphometrics_fold0_weighted.tif'
## output for fused image with morphometrics
# output_tif = r'imagery\hongkong_20180321_morphometrics_weighted_meanminmax_grid.tif'

### Paris
## image
# satellite_tif = r'imagery\paris_20170526.tif'
## rasterized morphometrics from non-weighted model
# secondary_tif = r'rasterized_morphometrics\paris_rasterized_morphometrics_fold1.tif'
## output for fused image with morphometrics
# output_tif = r'imagery\paris_20170526_morphometrics_meanminmax_grid.tif'

## image
# satellite_tif = r'imagery\paris_20170526.tif'
## rasterized morphometrics from weighted model
# secondary_tif = r'rasterized_morphometrics\paris_rasterized_morphometrics_fold1_weighted.tif'
## output for fused image with morphometrics
# output_tif = r'imagery\paris_20170526_morphometrics_weighted_meanminmax_grid.tif'

### Rome
## image
# satellite_tif = r'imagery\rome_20170620.tif'
## rasterized morphometrics from non-weighted model
# secondary_tif = r'rasterized_morphometrics\rome_rasterized_morphometrics_fold2.tif'
## output for fused image with morphometrics
# output_tif = r'imagery\rome_20170620_morphometrics_model_meanminmax_grid.tif'

## image
# satellite_tif = r'imagery\rome_20170620.tif'
## rasterized morphometrics from weighted model
# secondary_tif = r'rasterized_morphometrics\rome_rasterized_morphometrics_fold2_weighted.tif'
## output for fused image with morphometrics
# output_tif = r'imagery\rome_20170620_morphometrics_weighted_meanminmax_grid.tif'

### Sao Paulo
## image
# satellite_tif = r'imagery\sao_paulo_20170726.tif'
## rasterized morphometrics from non-weighted model
# secondary_tif = r'rasterized_morphometrics\saopaulo_rasterized_morphometrics_fold4.tif'
## output for fused image with morphometrics
# output_tif = r'imagery\sao_paulo_20170726_morphometrics_meanminmax_grid.tif'

## image
# satellite_tif = r'imagery\sao_paulo_20170726.tif'
## rasterized morphometrics from weighted model
# secondary_tif = r'rasterized_morphometrics\saopaulo_rasterized_morphometrics_fold1_weighted.tif'
## output for fused image with morphometrics
# output_tif = r'imagery\sao_paulo_20170726_morphometrics_weighted_meanminmax_grid.tif'

In [ ]:
stack_and_resample(
    satellite_tif,
    secondary_tif,
    output_tif,
    target_resolution=100
)